In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import xml.etree.ElementTree as ET
import numpy as np
import time
import os
import random
import xlwt
import string

from utils import *

## Load model and metadata from disk

In [2]:
model = SentenceTransformer('nlpaueb/legal-bert-base-uncased')
#model = SentenceTransformer('nlpaueb/bert-base-uncased-contracts')
meta_data_df = pd.read_excel('data/category_configuration_09-08-2022_08-08-01.xlsx', sheet_name = 'article_names_matching')
title2category = dict(zip(meta_data_df["Article Title"], meta_data_df["Category 2"]))

keyword_df = pd.read_excel('data/category_configuration_09-08-2022_08-08-01.xlsx', sheet_name = 'keywords_category_2_mapping')
#keyword2category2 = dict(zip(keyword_df["Keyword"], keyword_df["Category 2"]))
#keyword2category3 = dict(zip(keyword_df["Keyword"], keyword_df["Category 3"]))
keyword_df.dropna(subset = ["Keyword"], inplace=True)
keyword2category = {}
for i in range(len(keyword_df)):
    cat2 = keyword_df.iloc[i]['Category 2']
    cat3 = keyword_df.iloc[i]['Category 3']
    keyword = keyword_df.iloc[i]['Keyword']
    if keyword not in keyword2category:
        keyword2category[keyword] = {}
    if cat2 not in keyword2category[keyword]:
        keyword2category[keyword][cat2] = set()
    keyword2category[keyword][cat2].add(cat3)


No sentence-transformers model found with name C:\Users\Xiang/.cache\torch\sentence_transformers\nlpaueb_legal-bert-base-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\Xiang/.cache\torch\sentence_transformers\nlpaueb_legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactl

In [3]:
keyword_df[keyword_df['Keyword'] == 'formal requirement']

,Keyword,Category 4,Category 3,Category 2,Category 1,Additional Document Sections,Additional 2º level categories
474,formal requirement,Investor Information,Investor Information,Investor Information and Confidentiality,Non-Protection Provisions,NaN,NaN


## Code for calculate similarity between two documents

In [4]:
def get_article_alignment(all_articles1, all_articles2, title2category, sanity_check = True):
    '''
    Get article alignment between two documents
    article_body: xml element
    title2category: dictionary of title to category
    sanity_check: use similarity score to check if the alignment is correct
    return a dictionary of article alignment
    '''
    alignment_match = {}

    #filter out the arrticles that don't have title
    all_articles1 = [article for article in all_articles1 if article.get("title") is not None]
    all_articles2 = [article for article in all_articles2 if article.get("title") is not None]

    #if both articles attribute includes title
    #if 'title' in all_articles1[0].attrib and 'title' in all_articles2[0].attrib:
    article_title1 = [(article.get('title').lower(), article.get('num')) for article in all_articles1 if article.get('title')]
    article_title2 = [(article.get('title').lower(), article.get('num')) for article in all_articles2 if article.get('title')]
    for index1, (title1, num1) in enumerate(article_title1):
        for index2, (title2, num2) in enumerate(article_title2):
            if title2category.get(title1) == title2category.get(title2):
                #to make sure that they have a high similarity
                if sanity_check and max(util.cos_sim(
                    model.encode(''.join(elem2sent(all_articles1[index1]))),
                    model.encode(''.join(elem2sent(all_articles2[index2])))
                )).item() < 0.7:
                    continue
                alignment_match[num1] = num2
                break
                
    #title not included in article attri, use sentence similarity instead
    # else:
    #     for article1 in all_articles1:
    #         text1_embd = model.encode(''.join(elem2sent(article1)))
    #         text2_list_embd = model.encode([''.join(elem2sent(article2)) for article2 in all_articles2])
    #         scores = util.cos_sim(text1_embd, text2_list_embd)
    #         if max(scores[0]).item() > 0.7:
    #             index = np.argmax(scores[0])
    #             alignment_match[article1.get('num')] = all_articles2[index].get('num')

    return alignment_match
    

def extract_similar_sentences_from_article(article1, article2):
    '''
    article1, article2: xml element
    return: list of similar sentences and their category: (sentence1, sentence2, similarity), category
    '''
    if article1.get('title') and article2.get('title'):
        if (title2category.get(article1.get('title').lower()) != title2category.get(article2.get('title').lower())):
            return [], 'TITLE_MISMATCH'

    article1_sents, article2_sents = elem2sent(article1, break_sentence = False), elem2sent(article2, break_sentence = False)
    #tokenize the sentences
    #article1_sents = [model.tokenize(sent) for sent in article1_sents]


    #article1_sents, article2_sents = elem2sent(article1), elem2sent(article2)

    #Embed article1 and article2
    article1_embd, article2_embd = model.encode(article1_sents), model.encode(article2_sents)

    #Get similarity between article1 and article2
    scores = util.cos_sim(article1_embd, article2_embd)

    visited = set() #to make sure that we don't add the same sentence twice

    #filter out the sentence with similarity greater than 0.98, this means they are perfect match and no need to compare
    identical = (scores > 0.999).to(torch.int64)
    for i, j in identical.nonzero().tolist():
        visited.add('row' + str(i))
        visited.add('col' + str(j))

    #filter out the sentences with similarity between 0.5 and 0.98
    mask = (scores >= 0.91) & (scores < 1)
    scores *= mask.to(torch.int64) 

    #get the index of the sentences with similarity between 0.5 and 0.98
    sim_pairs = [(scores[i][j], i, j) for i, j in mask.nonzero().tolist()]
    sim_pairs.sort(key = lambda x: x[0]) #sort by similarity score
    
    ret = []
    while sim_pairs:
        score, i, j = sim_pairs.pop()
        if 'row' + str(i) not in visited and 'col' + str(j) not in visited:
            ret.append((article1_sents[i], article2_sents[j], scores[i][j].item()))
            visited.add('row' + str(i))
            visited.add('col' + str(j))

    return ret, title2category.get(article1.get('title').lower())

def get_cat3(sentence, keyword2category, cat2):
    '''
    sentence: string
    keyword2category: dictionary of keyword to category
    cat2: string
    return: category 3
    '''
    kws = set()
    res = set()
    sentence = sentence.lower().translate(str.maketrans('', '', string.punctuation))
    for keyword in keyword2category.keys():
        if keyword in sentence and cat2 in keyword2category[keyword]:
            kws.add(keyword)
            res |= keyword2category[keyword][cat2]
            #return keyword, keyword2category[keyword].get(cat2, set())
    return kws, res

def extract_similar_from_doc(doc1_path, doc2_path, title2category, target_category = [], min_length = 5):
    '''
    doc_path: path to first document
    title2category: dictionary of title to category
    '''

    try:   
        doc1, doc2 = ET.parse(doc1_path), ET.parse(doc2_path)
        doc_root1, doc_root2 = doc1.getroot(), doc2.getroot()
        doc_body1, doc_body2 = doc_root1[1][2], doc_root2[1][2]
        
    except Exception as e:
        print(e); return []
    
    #get article alignment between two documents
    all_articles1, all_articles2 = doc_body1.findall(".//div[@type='article']"), doc_body2.findall(".//div[@type='article']")
    alignment_match = get_article_alignment(all_articles1, all_articles2, title2category)

    ret = []
    for page1, page2 in alignment_match.items():
        article1, article2 = doc_body1.find(".//div[@num='" + page1 + "']"), doc_body2.find(".//div[@num='" + page2 + "']") 
        similar_sents, category2 = extract_similar_sentences_from_article(article1, article2)

        if category2 == 'TITLE_MISMATCH':
            continue

        if not target_category or category2 in target_category:
            for (sentence1, sentence2, score) in similar_sents:
                
                k1, s1_cate3 = get_cat3(sentence1, keyword2category, category2)
                k2, s2_cate3 = get_cat3(sentence2, keyword2category, category2)


                #filter out the pairs which has a category and which both sentences are longer than min_length words and length difference is less than 4 * min_length
                if (s1_cate3 and s2_cate3 and len(sentence1.split()) >= min_length and len(sentence2.split()) >= min_length and abs(len(sentence1) - len(sentence2)) <= 4 * min_length):
                    if s1_cate3 == s2_cate3 or s1_cate3 in s2_cate3 or s2_cate3 in s1_cate3:
                        if 0.965 < score < 0.985:
                            ret.append(('STYLYSTIC', sentence1, sentence2, score, list(s1_cate3)))
                    elif s1_cate3 != s2_cate3 and len(s1_cate3.intersection(s2_cate3)) <= 1/3 * len(s1_cate3.union(s2_cate3)):
                        ret.append(('RELEVANT', sentence1, sentence2, score, [list(s1_cate3), list(s2_cate3)]))

    #sort by similarity score
    ret.sort(key = lambda x: x[3], reverse = True)

    return ret

Sanaity Check

In [5]:
start = time.time()
diff = extract_similar_from_doc(
    'data/full data/t1989-9-canada-russian-federation-bit-1989.xml', 
    'data/full data/t1990-14-canada-czech-republic-bit-1990.xml',
    title2category,
    []
)
print(time.time() - start)

2.656249523162842


In [6]:
diff

[('STYLYSTIC',
  'any dispute between the contracting parties concerning the interpretation or application of this agreement shall whenever possible be settled through diplomatic channels',
  'any dispute between the contracting parties concerning the interpretation or application of this agreement shall whenever possible be settled amicably through consultations',
  0.9811272621154785,
  ['State-to-state Arbitration'])]

Filtering out the documents that contain the target category

In [7]:
docs = ['data/full data/' + _ for _ in os.listdir('data/full data') ]
print("total treaty is " + str(len(docs)))
target_category = ['Public Policy']
target_treaty = []

for treaty in docs:
    try:   
        doc1 = ET.parse(treaty)
        doc_root1 = doc1.getroot()
        doc_body1 = doc_root1[1][2]
        
    except Exception as e:
        continue

    for article in doc_body1:
        try:
            # if title2category.get(article.get("title").lower()) in target_category:
            if title2category.get(article.get("title").lower()):
                target_treaty.append(treaty)
                break
        except Exception as e:
            continue

print(len(target_treaty))
    

total treaty is 3309
2544


In [8]:
#Write all sentences to a excel file
wb = xlwt.Workbook()
ws = wb.add_sheet('sheet1')

#add header
ws.write(0, 0, 'label')
ws.write(0, 1, 'sSentence1')
ws.write(0, 2, 'sentence2')
ws.write(0, 3, 'similarity')
ws.write(0, 4, 'category3')
ws.write(0, 5, 'doc1')
ws.write(0, 6, 'doc2')


row = 1
res = []

visited = set()


total = 200
while total > 0:
    doc1 = random.choice(target_treaty)
    doc1name = doc1.replace('.','/').split('/')[2]

    doc2 = random.choice(target_treaty)
    doc2name = doc2.replace('.','/').split('/')[2]

    if (doc1, doc2) in visited or (doc2, doc1) in visited:
        continue
    visited.add((doc1, doc2))
    visited.add((doc2, doc1))

    # try:
    diff = extract_similar_from_doc(doc1, doc2, title2category, target_category = [])
    if any(diff):
        res.extend(diff)
        total -= len(diff)

        for i, (label, sentence1, sentence2, score, category3) in enumerate(diff):
            ws.write(row, 0, label)
            ws.write(row, 1, sentence1)
            ws.write(row, 2, sentence2)
            ws.write(row, 3, score)
            ws.write(row, 4, str(category3))
            ws.write(row, 5, doc1name)
            ws.write(row, 6, doc2name)
            row += 1
    
    # except Exception as e:
    #     print(e)
    #     print(doc1, doc2)
    #     # continue

wb.save('generated_data/similar_sentences.xlsx')


In [13]:
len(res)

200

In [14]:
sum([1 for _ in res if _[0] == 'STYLYSTIC'])

151

In [11]:
res

[('STYLYSTIC',
  'intellectual property rights including rights with respect to copyright patents trademarks trade names industrial designs trade secrets knowhow and goodwill and',
  'intellectual property rights including copyrights trademarks patents industrial designs technical processes knowhow trade secrets and trade names and goodwill and',
  0.9710628986358643,
  ['Investment Definition']),
 ('STYLYSTIC',
  ' returns means the amounts yielded by an investment and in particular though not exclusively includes profit interest capital gains dividends royalties and other fees ',
  'the term returns shall mean amounts yielded by an investment and in particular though not exclusively includes profits capital gains shares dividends royalties or fees',
  0.981878936290741,
  ['Investment Definition']),
 ('STYLYSTIC',
  'shares in and stock and debentures of a company and any other form of participation in a company',
  'shares stocks and debentures of companies or any other form of part

In [12]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(list(filter(None, corrected_text)))
        
text = "speling correctined heu"
correct_spellings(text)

'spelling corrected he'